# Test the Stream Index

## Check if index is up!

In [7]:
import requests
res = requests.get('http://localhost:8055')
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [8]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost'], port=8055)
print es

<Elasticsearch([{u'host': 'localhost', u'scheme': 'http'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [4]:
# es.indices.delete(index='stream', ignore=[400, 404])

{u'acknowledged': True}

In [105]:
# es.delete_by_query(index='stream', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

{u'batches': 1,
 u'deleted': 34,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 8,
 u'total': 34,
 u'version_conflicts': 0}

#### Count elements in index

In [11]:
es.count(index='stream', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 18522}

#### GET all docs from index

In [120]:
res = es.search(index="stream", doc_type='tweet', size=100, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:100]:
    if "ChiaraRenso" in r["_source"]["username"]  :
        print r
        print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'888057336996274176', u'_source': {u'username': u'ChiaraRenso', u'lang': u'und', u'text_categories': [], u'city': u'tokyo', u'text': u'#foodsigir2017 https://t.co/nCH5tezIhC', u'img_categories': [{u'score': 14.0, u'label': u'spaghetti_carbonara'}], u'hashtags': [u'#foodsigir2017'], u'img_flag': True, u'timestamp_ms': u'1500564343104', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[139.68794345855713, 35.66782614259895], [139.68760013580322, 35.662073226633424], [139.69326496124268, 35.65408818971974], [139.69918727874756, 35.64871789058102], [139.70167636871338, 35.64763681268489], [139.7023630142212, 35.64673009091096], [139.70725536346436, 35.6462069775141], [139.711332321167, 35.64903174916549], [139.7127056121826, 35.64854352416794], [139.71467971801755, 35.64840403076344], [139.71862792968747, 35.64892712977434], [139.71961498260498, 35.64871789058102], [139.72034454345703, 35.64826453711539], [139.72304821014404, 35.6476368

#### GET specific tweets

In [80]:
# es.get(index='stream', doc_type='tweet', id=880404048834416644)

In [66]:
results = es.search(index='stream', doc_type='tweet', size=3, body={
  'query': {
    'match': {
      'text_categories': 'donuts',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

print results
print len(results["hits"]["hits"])

{u'hits': {u'hits': [{u'_score': 4.1496673, u'_type': u'tweet', u'_id': u'885899203934814209', u'_source': {u'username': u'KenmareFoodies', u'lang': u'en', u'text_categories': [u'donuts'], u'city': None, u'text': u'Tuned in and waiting to hear @GastroGays talk all things food fad on @todayfm #donuts', u'img_categories': None, u'hashtags': [u'#donuts'], u'img_flag': True, u'timestamp_ms': u'1500049804086', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[-10.619545, 51.688185], [-10.619545, 52.602848], [-9.119114, 52.602848], [-9.119114, 51.688185]]]}, u'coords': None, u'id_str': u'885899203934814209', u'country': u'ireland', u'id': 885899203934814209, u'media_url': None, u'created_at_day': 20170714, u'created_at_datetime': u'2017-07-14T16:30:04', u'created_at_month': 20170701}, u'_index': u'stream'}, {u'_score': 4.1496673, u'_type': u'tweet', u'_id': u'885929891220381696', u'_source': {u'username': u'MichelleBargain', u'lang': u'en', u'text_categories': [u'donuts'], u'city': N

### Queries group by

In [73]:
import os
import sys
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from processing.load_keyword_dicts import loadCategoryDict


cats = set(loadCategoryDict().values())
print len(cats)
for x in cats:
    print x
    results = es.search(index='stream', doc_type='tweet', size=10000, body={
      'query': {
        'term': {
#             'img_flag': True,
            'img_categories.label': x,
#             'day': 20170629,
    #         'sort': [
    #         {'committed_date': {'order': 'desc'}}
    #       ],
         }
      }
    })
    print results["hits"]["total"]
#     if results["hits"]["total"]>0:
#         print results["hits"]["hits"]


101
grilled salmon
0
lobster roll sandwich
0
hamburger
28
churros
22
frozen yogurt
0
chicken quesadilla
0
samosa
7
pad thai
0
eggs benedict
0
peking duck
0
apple pie
0
sashimi
17
waffels
0
cup cakes
0
chicken curry
0
lobster bisque
0
cannoli
3
tacos
19
chocolate cake
0
poutine
1
ramen
29
bibimbap
5
beignets
0
risotto
5
chocolate mousse
0
paella
12
caesar salad
0
steak
14
spaghetti carbonara
0
beef tartare
0
falafel
12
chicken wings
0
club sandwich
0
ice cream
0
garlic bread
0
beef carpaccio
0
pork chop
0
escargots
2
tiramisu
3
fried rice
0
scallops
3
grilled cheese sandwich
0
baklava
1
edamame
4
carrot cake
0
pancakes
19
mussels
5
pulled pork sandwich
0
fried calamari
0
creme brulee
0
hummus
6
lasagna
4
red velvet cake
0
guacamole
11
clam chowder
0
foie gras
0
deviled eggs
0
beet salad
0
breakfast burrito
0
crab cakes
0
donuts
33
croque madame
0
gyoza
3
greek salad
0
ravioli
3
panna cotta
0
cheese plate
0
ceviche
13
spring rolls
0
bread pudding
0
bruschetta
6
shrimp and grits
0
french 

#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data